In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Model, Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam

from keras.datasets import mnist

In [49]:
def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = (x_train.astype(np.float32) - 127.5) / 127.5
    x_train = x_train.reshape(x_train.shape[0], 784)
    
    return (x_train, y_train, x_test, y_test)

In [50]:
x_train, y_train, x_test, y_test = load_data()
x_train.shape

(60000, 784)

In [51]:
def adam():
    adam = Adam(lr=0.0002, beta_1=0.5)
    return adam

In [52]:
def create_generator():    
    generator=Sequential()
    
    generator.add(Dense(units=256,input_dim=100))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=512))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=1024))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=784, activation='tanh'))
    
    generator.compile( loss='binary_crossentropy', optimizer=adam())
    
    return generator

g = create_generator()
g.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_329 (Dense)            (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_247 (LeakyReLU)  (None, 256)               0         
_________________________________________________________________
dense_330 (Dense)            (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_248 (LeakyReLU)  (None, 512)               0         
_________________________________________________________________
dense_331 (Dense)            (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_249 (LeakyReLU)  (None, 1024)              0         
_________________________________________________________________
dense_332 (Dense)            (None, 784)               803600    
Total para

In [53]:
def create_discriminator():    
    discriminator=Sequential()
    
    discriminator.add(Dense(units=1024,input_dim=784))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
       
    
    discriminator.add(Dense(units=512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
       
    discriminator.add(Dense(units=256))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(units=1, activation='sigmoid'))
    
    discriminator.compile(loss='binary_crossentropy', optimizer=adam())
    
    return discriminator

d =create_discriminator()
d.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_333 (Dense)            (None, 1024)              803840    
_________________________________________________________________
leaky_re_lu_250 (LeakyReLU)  (None, 1024)              0         
_________________________________________________________________
dropout_81 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_334 (Dense)            (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_251 (LeakyReLU)  (None, 512)               0         
_________________________________________________________________
dropout_82 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_335 (Dense)            (None, 256)               131328    
__________

In [54]:
def create_gan(discriminator, generator):    
    discriminator.trainable=False    
    gan_input = Input(shape=(100,))
    x = generator(gan_input)
    gan_output= discriminator(x)
    gan= Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    
    return gan

gan = create_gan(d,g)
gan.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        (None, 100)               0         
_________________________________________________________________
sequential_83 (Sequential)   (None, 784)               1486352   
_________________________________________________________________
sequential_84 (Sequential)   (None, 1)                 1460225   
Total params: 2,946,577
Trainable params: 1,486,352
Non-trainable params: 1,460,225
_________________________________________________________________


In [55]:
def plot_generated_images(epoch, generator, examples=100, dim=(10,10), figsize=(10,10)):    
    noise= np.random.normal(loc=0, scale=1, size=[examples, 100])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(100,28,28)
    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
  
    plt.savefig('gan_generated_image %d.png' %epoch)

In [56]:
def training(epochs=1, batch_size=128):    
    batch_count = X_train.shape[0] / batch_size
    
    generator = create_generator()
    discriminator = create_discriminator()
    gan = create_gan(discriminator, generator)
    
    for e in range(1, epochs+1 ):
        print("Epoch %d" %e)
        for _ in tqdm(range(batch_size)):
            noise= np.random.normal(0,1, [batch_size, 100])
            
            generated_images = generator.predict(noise)
            
            image_batch =X_train[np.random.randint(low=0,high=X_train.shape[0],size=batch_size)]
            
            X = np.concatenate([image_batch, generated_images])
            
            y_dis=np.zeros(2*batch_size)
            y_dis[:batch_size]=0.9

            discriminator.trainable=True
            discriminator.train_on_batch(X, y_dis)
            
            y_gen = np.ones(batch_size)
            
            discriminator.trainable=False
            gan.train_on_batch(noise, y_gen)
            
        if e == 1 or e % 20 == 0:
            plot_generated_images(e, generator)

In [ ]:
training(20)

  0%|          | 0/128 [00:00<?, ?it/s]

Epoch 1


  1%|          | 1/128 [00:00<00:20,  6.27it/s]

Epoch 2


  0%|          | 0/128 [00:00<?, ?it/s]

...

  1%|          | 1/128 [00:00<00:18,  6.76it/s]

Epoch 19


  1%|          | 1/128 [00:00<00:19,  6.55it/s]

Epoch 20


 25%|██▌       | 32/128 [00:04<00:16,  5.66it/s]